Manufacturing AI Agent

- Install Dependencies
  
```bash 
pipenv shell
pipenv install google-generativeai
pipenv install langchain langchain-google-genai
```

In [14]:
import os
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import HumanMessage
import google.generativeai as genai



/home/ozkary/.local/share/virtualenvs/chat-gpt-2H0-IQSn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# --- Setup ---
GEMINI_KEY = os.getenv("GEMINI_KEY")
# print(GEMINI_KEY)

# show available models
genai.configure(api_key=GEMINI_KEY)

for model in genai.list_models():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-

In [ ]:
# Initialize Gemini LLM

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,  # .1-.3 more deterministic
    google_api_key=GEMINI_KEY
)

# Initialize short-term memory (buffer of last 5 messages)
memory = ConversationBufferWindowMemory(k=5, return_messages=True)


In [12]:
# System prompt for structured agent behavior
SYSTEM_PROMPT = """
You are a specialized control chart monitoring agent in a manufacturing environment.
Your role is to observe new sensor readings and determine process stability using SPC (Statistical Process Control) logic.

Use this structured output:
@observe: Acknowledge the new reading.
@reason: Explain trend status or SPC rule violations, referencing the trend window.
@act: Recommend what action to take (e.g., monitor, alert supervisor, prep shutdown).
"""

# Sample input (you can swap this out for other sensor messages)
sensor_data = {
    "sensor": "vibration_motor_3A",
    "timestamp": "2025-06-18T11:47:09Z",
    "value": 0.89,
    "unit": "mm/s",
    "ucl": 0.90,
    "lcl": 0.50,
    "trend_window": [0.91, 0.94, 0.86, 0.89],
    "alert_enabled": True
}

# Prepare prompt
formatted_data = json.dumps(sensor_data, indent=2)
prompt = f"{SYSTEM_PROMPT}\n\nNew telemetry data:\n{formatted_data}"


In [24]:
from IPython.display import display, Markdown
def display_agent_response(response): 
    print("===== 🤖 AI AGENT RESPONSE =====\n")
    try:
        # Convert string to dictionary
        response_dict = json.loads(response)           
        print(f"\n📡 @observe:\n{response_dict.get('@observe', 'N/A')}")
        print(f"\n🧠 @reason:\n{response_dict.get('@reason', 'N/A')}")
        print(f"\n⚙️ @act:\n{response_dict.get('@act', 'N/A')}")
        print("\n" + "="*30)
    except Exception as e:
         display(Markdown(response))


In [25]:
# Run LLM with memory context
response = llm.invoke([HumanMessage(content=prompt)])

# Save exchange to memory
memory.save_context({"input": formatted_data}, {"output": response.content})

# Print the response
display_agent_response(response.content)

===== 🤖 AI AGENT RESPONSE =====



```json
{
  "@observe": "Observed new vibration reading for motor 3A: 0.89 mm/s.",
  "@reason": "The current reading (0.89 mm/s) is within the control limits (UCL=0.9 mm/s, LCL=0.5 mm/s). However, the last four readings in the trend window are [0.91, 0.94, 0.86, 0.89]. The two most recent readings are showing an upward trend. While not a violation of standard SPC rules yet, the two previous readings were above the UCL. This warrants close monitoring.",
  "@act": "Monitor the vibration levels closely for the next few readings. If the upward trend continues or the UCL is breached again, alert the supervisor."
}
```